In [1]:
import cv2
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
from pymongo import MongoClient
from torchvision import transforms
from PIL import Image

In [2]:
mongo_client = MongoClient("mongodb://localhost:27017/")
db = mongo_client["faceRecog"]
collection = db["faceData"]

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
class FaceRecogModel(nn.Module):
    def __init__(self):
        super(FaceRecogModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)

        self.linear_dims = None
        x = torch.randn(1, 3, 128, 128)
        self.convs(x)
        self.fc1 = nn.Linear(self.linear_dims, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.flatten = nn.Flatten()

    def convs(self ,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        if self.linear_dims is None:
            self.linear_dims = x.shape[1]*x.shape[2]*x.shape[3]
            print(self.linear_dims)
        return x

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.flatten(x)
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = torch.load('model_weights.pt')
model.to(device)
model.eval()

FaceRecogModel(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=13456, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)

In [27]:
documents = collection.find()
database = []

for doc in documents:
    name = doc['name']
    face_vector = doc['face_vector']
    tensor = torch.tensor(face_vector).to(device)
    data = {'name': name, 'face_vector': tensor}
    database.append(data)

In [6]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [28]:
lst = []
tens = []
def predictFace(face):
    imgarr = Image.fromarray(face)
    tensor_image = transform(imgarr).unsqueeze(0).cuda()
    output = model(tensor_image)

    current_similarity = 0
    name = ""
    for data in database:
        similarity = F.cosine_similarity(output, data["face_vector"])
        if similarity > current_similarity:
            current_similarity = similarity
            name = data["name"]

    if current_similarity > 0.9:
        lst.append(current_similarity)
        tens.append(output)
        return name
    else:
        return "Unknown"

In [29]:
cap = cv2.VideoCapture(0)
cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
font = cv2.FONT_HERSHEY_COMPLEX
text = ""

while True:
    
    ret, frame = cap.read()
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    
    border = cascade.detectMultiScale(gray)

    if len(border) == 0:
        text = "No Face Detected"
     
    if len(border) > 1:
        text = "Multiple Faces Detected"
    
    for (x, y, w, h) in border:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), thickness=2)
        ROI = frame[y:y+h, x:x+w]
        text = predictFace(ROI)
    
    if text != "":
        cv2.putText(frame, text, (frame.shape[0]//2,frame.shape[1]//2), font, 1, (0,0,255), 2)

    cv2.imshow("frame", frame)
    text = ""
    k = cv2.waitKey(1)
    if k%256==27:
        break

        
cap.release()
cv2.destroyAllWindows()